<a href="https://colab.research.google.com/github/samaneh-m/TU-simulation-base-inference/blob/main/forward_backward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "bayesflow>=2.0"
!pip install tensorflow hmmlearn  # Add hmmlearn for HMM inference

In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import bayesflow as bf
import numpy as np
import tensorflow as tf
from hmmlearn import hmm

from tensorflow.keras.utils import to_categorical

# Amino‑acid mapping
amino_acids = ['A','R','N','D','C','E','Q','G','H','I',
               'L','K','M','F','P','S','T','W','Y','V']
aa_to_int = {aa:i for i, aa in enumerate(amino_acids)}

class HMMHiddenStateSimulator:
    def __init__(self, seq_len=50):
        self.seq_len = seq_len
        self.transmat = np.array([[0.90, 0.10],[0.05, 0.95]])
        self.startprob = np.array([0.0, 1.0])
        emissions = {
            'alpha': np.array([12,6,3,5,1,9,5,4,2,7,12,6,3,4,2,5,4,1,3,6]) / 100,
            'other': np.array([6,5,5,6,2,5,3,9,3,5,8,6,2,4,6,7,6,1,4,7]) / 100
        }
        self.emissionprob = np.stack([emissions['alpha'], emissions['other']])
        # ✅ Use categorical HMM for integer observations
        self.model = hmm.CategoricalHMM(n_components=2, n_iter=100, init_params="")
        self.model.startprob_ = self.startprob
        self.model.transmat_ = self.transmat
        self.model.emissionprob_ = self.emissionprob

    def sample(self, n_sequences=1000):
        X = []
        for _ in range(n_sequences):
            seq, _ = self.model.sample(n_samples=self.seq_len)
            X.append(seq.flatten())
        X_all = np.concatenate(X).reshape(-1, 1)
        post_probs = self.model.predict_proba(X_all)
        theta_post = post_probs[:, 0].reshape(n_sequences, self.seq_len)
        x_obs = np.array(X).reshape(n_sequences, self.seq_len)
        return x_obs, theta_post

# Simulate data & encode
sim = HMMHiddenStateSimulator(seq_len=50)
x_train, theta_train = sim.sample(n_sequences=1000)
x_train_onehot = to_categorical(x_train, num_classes=20)

class ReplaySimulator:
    def __init__(self, x_data, theta_data):
        self.x = x_data
        self.theta = theta_data
        self.n = theta_data.shape[0]

    def sample(self, batch_size):
        idx = np.random.choice(self.n, size=batch_size, replace=False)
        return {"hidden_states": self.theta[idx], "observations": self.x[idx]}

# BayesFlow setup
N, seq_len, AA = x_train_onehot.shape

summary_net = bf.networks.TimeSeriesNetwork(
    input_shape=(seq_len, AA),
    summary_variables=["observations"],
    layers=[128, 64],
    rnn_units=64
)

inference_net = bf.networks.CouplingFlow(
    n_parameters=seq_len,
    context_summary=["observations"],
    coupling_layers=6,
    hidden_sizes=[128, 128]
)

simulator = ReplaySimulator(x_train_onehot, theta_train)

workflow = bf.BasicWorkflow(
    inference_network=inference_net,
    summary_network=summary_net,
    inference_variables=["hidden_states"],
    summary_variables=["observations"],
    simulator=simulator
)

# ✅ Train with soft posteriors instead of hard labels
history = workflow.fit_online(
    epochs=10,
    batch_size=64,
    num_batches_per_epoch=200
)


INFO:bayesflow:Fitting on dataset instance of OnlineDataset.
INFO:bayesflow:Building on a test batch.


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 70s 152ms/step - loss: 13.5312
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 157ms/step - loss: -24.5958
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 159ms/step - loss: -31.3726
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 30s 150ms/step - loss: -36.9478
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 159ms/step - loss: -41.4356
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 42s 162ms/step - loss: -45.1748
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 46s 189ms/step - loss: -48.3336
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 160ms/step - loss: -50.8052
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 159ms/step - loss: -52.3762
Epoch 10/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 33s 165ms/step - loss: -53.3093


In [5]:
# ✅ Train with soft posteriors instead of hard labels
history2 = workflow.fit_online(
    epochs=20,
    batch_size=64,
    num_batches_per_epoch=200
)

INFO:bayesflow:Fitting on dataset instance of OnlineDataset.


Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 154ms/step - loss: -53.3342
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 160ms/step - loss: -53.1325
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 162ms/step - loss: -53.2552
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 156ms/step - loss: -53.1871
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 155ms/step - loss: -53.2458
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 42s 160ms/step - loss: -53.3168
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 155ms/step - loss: -53.1924
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 153ms/step - loss: -53.2347
Epoch 9/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - loss: -53.3129
Epoch 10/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 154ms/step - loss: -53.3197
Epoch 11/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 33s 164ms/step - loss: -53.3273
Epoch 12/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 158ms/step - loss: -53.2743
Epoch 13/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 32s 162ms/step - loss: -53.3006
Epoch 14/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 161ms/step - lo